In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
df_1 = pd.read_csv('/content/sales_data_january_2019.csv')
df_2 = pd.read_csv('/content/sales_data_february_2019.csv')
df_3 = pd.read_csv('/content/sales_data_march_2019.csv')
df_4 = pd.read_csv('/content/sales_data_april_2019.csv')
df_5 = pd.read_csv('/content/sales_data_may_2019.csv')
df_6 = pd.read_csv('/content/sales_data_june_2019.csv')
df_7 = pd.read_csv('/content/sales_data_july_2019.csv')
df_8 = pd.read_csv('/content/sales_data_august_2019.csv')
df_9 = pd.read_csv('/content/sales_data_september_2019.csv')
df_10 = pd.read_csv('/content/sales_data_october_2019.csv')
df_11 = pd.read_csv('/content/sales_data_november_2019.csv')
df_12 = pd.read_csv('/content/sales_data_december_2019.csv')

In [ ]:
df = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10, df_11, df_12], ignore_index=True)
df.head(12)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,141234,iPhone,1,700,01/22/19 21:25,"944 Walnut St, Boston, MA 02215"
1,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,"185 Maple St, Portland, OR 97035"
2,141236,Wired Headphones,2,11.99,01/17/19 13:33,"538 Adams St, San Francisco, CA 94016"
3,141237,27in FHD Monitor,1,149.99,01/05/19 20:33,"738 10th St, Los Angeles, CA 90001"
4,141238,Wired Headphones,1,11.99,01/25/19 11:59,"387 10th St, Austin, TX 73301"
5,141239,AAA Batteries (4-pack),1,2.99,01/29/19 20:22,"775 Willow St, San Francisco, CA 94016"
6,141240,27in 4K Gaming Monitor,1,389.99,01/26/19 12:16,"979 Park St, Los Angeles, CA 90001"
7,141241,USB-C Charging Cable,1,11.95,01/05/19 12:04,"181 6th St, San Francisco, CA 94016"
8,141242,Bose SoundSport Headphones,1,99.99,01/01/19 10:30,"867 Willow St, Los Angeles, CA 90001"
9,141243,Apple Airpods Headphones,1,150,01/22/19 21:20,"657 Johnson St, San Francisco, CA 94016"


In [ ]:
# Step 1 & 2: Eksplorasi dan Pembersihan Data
# Drop missing rows
df_clean = df.dropna()

In [ ]:
# Lakukan proses konversi kolom 'Quantity Ordered' dan 'Price Each' ke tipe data numerik
# Gunakan errors='coerce' untuk mengubah nilai yang tidak bisa dikonversi menjadi NaN
df_clean['Quantity Ordered'] = pd.to_numeric(df_clean['Quantity Ordered'], errors='coerce')  # Mengubah nilai ke angka, jika error jadi NaN
df_clean['Price Each'] = pd.to_numeric(df_clean['Price Each'], errors='coerce')  # Sama seperti di atas

/tmp/ipython-input-3411546557.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Quantity Ordered'] = pd.to_numeric(df_clean['Quantity Ordered'], errors='coerce')  # Mengubah nilai ke angka, jika error jadi NaN
/tmp/ipython-input-3411546557.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Price Each'] = pd.to_numeric(df_clean['Price Each'], errors='coerce')  # Sama seperti di atas


In [ ]:
# Hapus baris yang memiliki nilai kosong (NaN) setelah konversi ke numerik
# Biasanya terjadi jika ada data yang salah format, misalnya huruf di kolom angka
df_clean = df_clean.dropna(subset=['Quantity Ordered', 'Price Each'])  # Hanya hapus baris yang kolom ini kosong
# Tampilkan satu baris pertama untuk memastikan hasilnya bersih
df_clean.head(1)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,141234,iPhone,1.0,700.0,01/22/19 21:25,"944 Walnut St, Boston, MA 02215"


In [ ]:
# Ubah kolom 'Order Date' menjadi format datetime
# Gunakan errors='coerce' agar data yang tidak bisa dikonversi (misalnya format salah) menjadi NaT (Not a Time)
df_clean['Order Date'] = pd.to_datetime(df_clean['Order Date'], errors='coerce')
df_clean.head(1)

/tmp/ipython-input-4129842114.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_clean['Order Date'] = pd.to_datetime(df_clean['Order Date'], errors='coerce')


,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,141234,iPhone,1.0,700.0,2019-01-22 21:25:00,"944 Walnut St, Boston, MA 02215"


In [ ]:
# Hapus baris yang memiliki 'Order Date' kosong atau tidak valid (NaT)
# Ini biasanya hasil dari kesalahan format saat konversi ke datetime
df_clean = df_clean.dropna(subset=['Order Date'])

In [ ]:
# Tambahkan kolom baru bernama 'Revenue' (total penjualan per baris transaksi)
# Didapat dari mengalikan jumlah barang yang dipesan dengan harga per item
df_clean['Revenue'] = df_clean['Quantity Ordered'] * df_clean['Price Each']
df_clean.head(1)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Revenue
0,141234,iPhone,1.0,700.0,2019-01-22 21:25:00,"944 Walnut St, Boston, MA 02215",700.0


In [ ]:
# # Pastikan 'Order Date' sudah dalam format datetime
# df_clean['Order Date'] = pd.to_datetime(df_clean['Order Date'], errors='coerce')

# Setelah yakin datetime, baru ambil tanggal saja (tanpa jam) dan lakukan agregasi
daily_revenue = df_clean.groupby(df_clean['Order Date'].dt.date)['Revenue'].sum().reset_index()

# Rename kolom agar lebih deskriptif
daily_revenue.columns = ['Date', 'Total_Revenue']

# Konversi kembali kolom 'Date' ke datetime agar bisa dipakai untuk plotting atau time series model
daily_revenue['Date'] = pd.to_datetime(daily_revenue['Date'])

# Tampilkan hasilnya
daily_revenue.head(1)

,Date,Total_Revenue
0,2019-01-01,65681.94


In [ ]:
# Atur kolom 'Date' sebagai index dari DataFrame
# Ambil hanya kolom 'Total_Revenue' sehingga hasilnya menjadi time series (seri waktu) murni

ts = daily_revenue.set_index('Date')['Total_Revenue']
ts.head(1)

,Total_Revenue
Date,
2019-01-01,65681.94


In [ ]:
df_clean.dtypes

,0
Order ID,object
Product,object
Quantity Ordered,float64
Price Each,float64
Order Date,datetime64[ns]
Purchase Address,object
Revenue,float64


In [ ]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185950 entries, 0 to 186849
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Order ID          185950 non-null  object        
 1   Product           185950 non-null  object        
 2   Quantity Ordered  185950 non-null  float64       
 3   Price Each        185950 non-null  float64       
 4   Order Date        185950 non-null  datetime64[ns]
 5   Purchase Address  185950 non-null  object        
 6   Revenue           185950 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 11.3+ MB


In [ ]:
df_clean.isna().sum()

,0
Order ID,0
Product,0
Quantity Ordered,0
Price Each,0
Order Date,0
Purchase Address,0
Revenue,0


In [ ]:
# Pastikan kolom datetime sudah rapi (kalau belum dilakukan sebelumnya)
df_clean['Order Date'] = pd.to_datetime(df_clean['Order Date'], errors='coerce')
df_clean = df_clean.dropna(subset=['Order Date'])

# Tambahkan kolom hari dan bulan
df_clean['weekday'] = df_clean['Order Date'].dt.day_name()
df_clean['month'] = df_clean['Order Date'].dt.month_name()

In [ ]:
# Agregasi rata-rata revenue per hari dalam seminggu
revenue_by_weekday = df_clean.groupby('weekday')['Revenue'].mean().reindex([
    'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'
])

# Tampilkan hasil numerik
print('Rata-rata revenue per hari:')
print(revenue_by_weekday)

Rata-rata revenue per hari:
weekday
Monday       183.950229
Tuesday      187.229320
Wednesday    188.420979
Thursday     182.890486
Friday       185.009273
Saturday     185.125963
Sunday       185.762105
Name: Revenue, dtype: float64


In [ ]:
# Tambahkan kolom minggu ke berapa dalam tahun (ISO week)
df_clean['week'] = df_clean['Order Date'].dt.isocalendar().week

# Agregasi revenue mingguan
weekly_revenue = df_clean.groupby('week')['Revenue'].sum().sort_index()

# Tampilkan hasil
print('Total revenue per minggu:')
print(weekly_revenue)

Total revenue per minggu:
week
1      637300.07
2      409428.28
3      394936.90
4      426632.16
5      460355.55
6      566203.88
7      568227.10
8      535323.33
9      578418.38
10     640535.27
11     596595.99
12     629763.90
13     671432.83
14     774904.91
15     761676.00
16     805556.11
17     808871.58
18     764849.44
19     729427.91
20     695770.71
21     711496.54
22     668258.10
23     616391.11
24     594326.71
25     612010.28
26     577539.85
27     603371.29
28     610101.51
29     615510.31
30     563148.08
31     543607.49
32     550752.14
33     493178.46
34     490623.22
35     489864.53
36     478816.84
37     537375.76
38     434949.27
39     506249.62
40     799212.50
41     826241.75
42     846999.42
43     847769.59
44     811488.35
45     747093.90
46     713430.04
47     775799.61
48     788703.80
49    1048487.00
50    1039123.69
51    1074953.03
52    1018951.88
Name: Revenue, dtype: float64


In [ ]:
# Agregasi total revenue per bulan
from calendar import month_name
revenue_by_month = df_clean.groupby('month')['Revenue'].sum()

# Reorder agar sesuai urutan Januari–Desember
ordered_months = list(month_name)[1:]
revenue_by_month = revenue_by_month.reindex(ordered_months)

# Tampilkan hasil numerik
print('Total revenue per bulan:')
print(revenue_by_month)

Total revenue per bulan:
month
January      1822256.73
February     2202022.42
March        2807100.38
April        3390670.24
May          3152606.75
June         2577802.26
July         2647775.76
August       2244467.88
September    2097560.13
October      3736726.88
November     3199603.20
December     4613443.34
Name: Revenue, dtype: float64


Revenue lebih tinggi saat hari kerja dan mencapai puncak di Desember menunjukkan pola aktivitas belanja aktif saat weekday dan lonjakan musim liburan. Tren mingguan juga meningkat menjelang akhir tahun, membuka peluang strategi promosi mingguan

In [ ]:
# Filter data 3 bulan terakhir (misalnya: Okt–Des)
df_q4 = df_clean[df_clean['Order Date'].dt.month.isin([10, 11, 12])]

# Hitung revenue per produk
top_products = df_q4.groupby('Product')['Revenue'].sum().sort_values(ascending=False).head(10)
print(top_products)

Product
Macbook Pro Laptop            2733600.00
iPhone                        1600900.00
ThinkPad Laptop               1373986.26
Google Phone                  1083000.00
27in 4K Gaming Monitor         843158.38
Apple Airpods Headphones       789000.00
34in Ultrawide Monitor         786959.29
Flatscreen TV                  495000.00
Bose SoundSport Headphones     454054.59
27in FHD Monitor               370775.28
Name: Revenue, dtype: float64


In [ ]:
# Ekstrak nama kota dari alamat
df_clean['City'] = df_clean['Purchase Address'].apply(lambda x: x.split(',')[1].strip())

# Order terbanyak
top_cities_by_order = df_clean['City'].value_counts().head(5)

# Spending terbesar
city_revenue = df_clean.groupby('City')['Revenue'].sum().sort_values(ascending=False).head(5)

# Rata-rata spending
city_avg_spending = df_clean.groupby('City')['Revenue'].mean().sort_values(ascending=False).head(5)

print("Top 5 Kota berdasarkan Jumlah Order:")
print(top_cities_by_order)

print("Top 5 Kota berdasarkan Total Revenue:")
print(city_revenue)

print("Top 5 Kota berdasarkan Rata-rata Spending:")
print(city_avg_spending)

Top 5 Kota berdasarkan Jumlah Order:
City
San Francisco    44732
Los Angeles      29605
New York City    24876
Boston           19934
Atlanta          14881
Name: count, dtype: int64
Top 5 Kota berdasarkan Total Revenue:
City
San Francisco    8262203.91
Los Angeles      5452570.80
New York City    4664317.43
Boston           3661642.01
Atlanta          2795498.58
Name: Revenue, dtype: float64
Top 5 Kota berdasarkan Rata-rata Spending:
City
Atlanta          187.856903
New York City    187.502711
Dallas           186.772969
Seattle          186.516120
Portland         186.160498
Name: Revenue, dtype: float64


In [ ]:
summary_df = pd.DataFrame({
    'Total Orders': top_cities_by_order,
    'Total Revenue': city_revenue,
    'Average Spending': city_avg_spending
})
print(summary_df)

               Total Orders  Total Revenue  Average Spending
City                                                        
Atlanta             14881.0     2795498.58        187.856903
Boston              19934.0     3661642.01               NaN
Dallas                  NaN            NaN        186.772969
Los Angeles         29605.0     5452570.80               NaN
New York City       24876.0     4664317.43        187.502711
Portland                NaN            NaN        186.160498
San Francisco       44732.0     8262203.91               NaN
Seattle                 NaN            NaN        186.516120


In [ ]:
# Tambah kolom jam transaksi
df_clean['Hour'] = df_clean['Order Date'].dt.hour

# Hitung jumlah order per jam
orders_by_hour = df_clean.groupby('Hour')['Order ID'].count().sort_values(ascending=False)
print("Jumlah order per jam (urut dari paling tinggi):")
print(orders_by_hour.head())

Jumlah order per jam (urut dari paling tinggi):
Hour
19    12905
12    12587
11    12411
18    12280
20    12228
Name: Order ID, dtype: int64


Puncak aktivitas belanja terjadi pada jam 19:00, diikuti oleh jam 12:00 dan 11:00

Pola jam transaksi menunjukkan dua momen utama belanja :

Makan siang (11:00 - 12:00)

Setelah jam kerja (18:00 - 20:00)

Ini mengindikasikan bahwa waktu senggang baik siang maupun malam mendorong konsumen untuk berbelanja

Rekomendasi Bisnis Berdasarkan Insight :

Promosi jam malam (Flash Sale pukul 19:00) bisa ditargetkan ke pengguna yang baru selesai bekerja

Push notif atau kampanye digital sekitar jam makan siang juga efektif menjangkau pengguna yang aktif saat istirahat

In [ ]:
# Lakukan uji ADF (Augmented Dickey-Fuller) untuk mengecek apakah data time series stasioner atau tidak
# Data dikatakan stasioner jika p-value < 0.05 (artinya tidak ada akar unit/root)

adf_result = adfuller(ts)  # Jalankan ADF Test
print(f'ADF Statistic: {adf_result[0]}')  # Nilai statistik ADF (semakin kecil = lebih stasioner)
print(f'p-value: {adf_result[1]}')        # p-value hasil uji, jika < 0.05 berarti data stasioner

ADF Statistic: -2.3508591590746564
p-value: 0.1560888501412367


In [ ]:
# Jika data tidak stasioner (p-value > 0.05), maka kita perlu mentransformasi datanya

# Langkah 1: Terapkan log transform untuk menstabilkan varians (mengurangi efek lonjakan besar)
ts_log = np.log(ts)

# Langkah 2: Terapkan differencing (pengurangan nilai dengan nilai sebelumnya)
# Tujuannya untuk menghilangkan tren agar data menjadi lebih stasioner
ts_log_diff = ts_log.diff().dropna()  # .diff() akan menghasilkan NaN di baris pertama, maka gunakan .dropna()

In [ ]:
# Lakukan uji ADF lagi setelah transformasi log dan differencing
# Tujuannya untuk memastikan data sudah menjadi stasioner

adf_result_diff = adfuller(ts_log_diff)  # Jalankan ADF Test pada data yang telah ditransformasi
print(f'ADF Statistic after diff: {adf_result_diff[0]}')  # Statistik ADF baru
print(f'p-value after diff: {adf_result_diff[1]}')        # p-value baru, harus < 0.05 untuk stasioner

ADF Statistic after diff: -9.433322671417482
p-value after diff: 5.12035581943102e-16


In [ ]:
# Pisahkan data menjadi dua bagian: data latih (train) dan data uji (test)
# Di sini, kita gunakan data Februari untuk melatih model
# Dan data Maret digunakan sebagai validasi untuk menguji performa model

train = ts[:'2019-02-28']   # Data latih: sampai akhir Februari
test = ts['2019-03-01':'2019-03-31'] # Data uji: mulai dari 1 Maret ke depan

4.  Model Forecasting

In [ ]:
# Buat model ARIMA untuk data time series
# Di sini kita gunakan log(train) agar model lebih stabil terhadap variasi besar (log transform)
# Parameter (1,1,1) artinya: p=1 (Auto REGRESSION), d=1 (differencing), q=1 (MOVING AVERAGE)

model_arima = ARIMA(np.log(train), order=(1,1,1))  # Buat model ARIMA dengan log-transformed data
result_arima = model_arima.fit()  # Latih model ARIMA

# Forecast dalam bentuk log selama panjang data uji (misal Maret = 31 hari)
forecast_arima_log = result_arima.forecast(steps=len(test))

# Kembalikan hasil forecast dari skala log ke skala asli dengan fungsi eksponensial
forecast_arima = np.exp(forecast_arima_log)  # Ini adalah hasil prediksi revenue harian

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


In [ ]:
# Buat model ETS (Exponential Smoothing) dengan komponen tren dan musiman
# 'trend="add"' artinya kita mengasumsikan adanya tren naik/turun yang bersifat aditif
# 'seasonal="add"' artinya ada pola musiman (berulang) yang juga ditambahkan
# 'seasonal_periods=7' menunjukkan adanya pola mingguan (7 hari)

model_ets = ExponentialSmoothing(train, trend='add', seasonal='add', seasonal_periods=7)  # Buat model ETS
result_ets = model_ets.fit()  # Latih modelnya

# Lakukan prediksi sebanyak panjang data uji
forecast_ets = result_ets.forecast(steps=len(test))  # Hasil prediksi revenue harian dari model ETS

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


In [ ]:
# Hitung MAPE (Mean Absolute Percentage Error) untuk mengukur akurasi kedua model
# Semakin kecil nilai MAPE, semakin akurat prediksi model terhadap data aktual

mape_arima = mean_absolute_percentage_error(test, forecast_arima)  # Error ARIMA
mape_ets = mean_absolute_percentage_error(test, forecast_ets)      # Error ETS

# Tampilkan hasil evaluasi dalam format persentase
print(f'MAPE ARIMA: {mape_arima:.2%}')  # Misalnya: 8.45%
print(f'MAPE ETS: {mape_ets:.2%}')

MAPE ARIMA: 14.89%
MAPE ETS: 8.14%


In [ ]:
# Bandingkan nilai MAPE dari ARIMA dan ETS
# Pilih model dengan MAPE terkecil sebagai model terbaik

best_model = "ARIMA" if mape_arima < mape_ets else "ETS"  # Jika MAPE ARIMA lebih kecil, pilih ARIMA, jika tidak pilih ETS

# Tampilkan model terbaik berdasarkan evaluasi MAPE
print(f"Model terbaik berdasarkan MAPE adalah: {best_model}")

Model terbaik berdasarkan MAPE adalah: ETS


In [ ]:
# Forecast ulang selama 30 hari (1 bulan ke depan) menggunakan model terbaik yang sudah ditentukan
from sklearn.metrics import mean_absolute_error, mean_squared_error

steps_ahead = 30  # Prediksi 30 hari ke depan

if best_model == "ARIMA":
    # Fit ulang ARIMA ke seluruh data latih
    tuned_arima_model = ARIMA(np.log(train), order=(1,1,1)).fit()

    # Prediksi 30 hari ke depan dalam skala log
    forecast_log = tuned_arima_model.forecast(steps=steps_ahead)

    # Kembalikan hasil ke skala asli
    forecast = np.exp(forecast_log)

else:
    # Fit ulang ETS ke seluruh data latih
    tuned_ets_model = ExponentialSmoothing(train, trend='add', seasonal='add', seasonal_periods=7).fit()

    # Prediksi 30 hari ke depan
    forecast = tuned_ets_model.forecast(steps=steps_ahead)

# Siapkan tanggal untuk hasil prediksi
forecast.index = pd.date_range(start='2019-04-01', periods=steps_ahead, freq='D')

# Jika kamu memiliki actual test data 30 hari juga (misal Maret), evaluasi bisa dilakukan
# Kalau tidak, langkah evaluasi di bawah hanya ilustratif
# Ubah test ke test_30 jika tersedia, atau sesuaikan:
test_30 = test[:steps_ahead]  # Ambil 30 hari pertama dari test untuk evaluasi

# Hitung metrik evaluasi
mape = mean_absolute_percentage_error(test_30, forecast)
mae = mean_absolute_error(test_30, forecast)
rmse = np.sqrt(mean_squared_error(test_30, forecast))

# Tampilkan hasil evaluasi
print(f"MAPE: {mape:.2%}")
print(f"MAE : {mae:,.2f}")
print(f"RMSE: {rmse:,.2f}")

MAPE: 8.36%
MAE : 7,783.18
RMSE: 9,401.81


/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


In [ ]:
steps_ahead = 30

# Forecast berdasarkan model terbaik
if best_model == "ARIMA":
    tuned_arima_model = ARIMA(np.log(ts), order=(1, 1, 1)).fit()
    forecast_log = tuned_arima_model.forecast(steps=steps_ahead)
    forecast_april = np.exp(forecast_log)
else:
    forecast_april = result_ets.forecast(steps=steps_ahead)

In [ ]:
forecast_april_df = forecast_april.to_frame().reset_index()
forecast_april_df.columns = ['Date', 'Forecasted_Revenue']
forecast_april_df.head()

,Date,Forecasted_Revenue
0,2019-03-01,82835.661571
1,2019-03-02,78656.536196
2,2019-03-03,82186.726423
3,2019-03-04,77248.977105
4,2019-03-05,82373.792798


In [ ]:
# Mengubah hasil prediksi time series menjadi DataFrame biasa dengan kolom terpisah
# forecast_april awalnya berupa pandas Series, dengan index = tanggal dan value = revenue
# .to_frame() akan mengubah Series menjadi DataFrame
# .reset_index() akan memindahkan index (tanggal) ke kolom biasa agar lebih mudah dilihat/digunakan

forecast_april_df = forecast_april.to_frame().reset_index()

# Ganti nama kolom agar lebih deskriptif dan rapi
forecast_april_df.columns = ['Date', 'Forecasted_Revenue']

# Tampilkan beberapa baris teratas sebagai preview
forecast_april_df.head()

,Date,Forecasted_Revenue
0,2019-03-01,82835.661571
1,2019-03-02,78656.536196
2,2019-03-03,82186.726423
3,2019-03-04,77248.977105
4,2019-03-05,82373.792798


In [ ]:
# Hitung jumlah visitor unik per hari (bisa pakai jumlah Order ID unik)
daily_visitor = df_clean.groupby(df_clean['Order Date'].dt.date)['Order ID'].nunique().reset_index()
daily_visitor.columns = ['Date', 'Visitor_Count']
daily_visitor['Date'] = pd.to_datetime(daily_visitor['Date'])

In [ ]:
ts_visitor = daily_visitor.set_index('Date')['Visitor_Count']

In [ ]:
# Kalau datanya belum stasioner, lakukan log transform dan differencing
ts_visitor_log = np.log(ts_visitor)
ts_visitor_log_diff = ts_visitor_log.diff().dropna()

In [ ]:
# ARIMA Model untuk Visitor
model_arima_visitor = ARIMA(np.log(ts_visitor), order=(1, 1, 1)).fit()
forecast_arima_log_v = model_arima_visitor.forecast(steps=30)
forecast_arima_visitor = np.exp(forecast_arima_log_v)

# ETS Model untuk Visitor
model_ets_visitor = ExponentialSmoothing(ts_visitor, trend='add', seasonal='add', seasonal_periods=7).fit()
forecast_ets_visitor = model_ets_visitor.forecast(steps=30)

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


In [ ]:
# Bagi dulu data visitor ke train dan test (misal: Feb & Mar)
train_v = ts_visitor[:'2019-02-28']
test_v = ts_visitor['2019-03-01':]

# Evaluasi akurasi prediksi
mape_arima_v = mean_absolute_percentage_error(test_v, np.exp(model_arima_visitor.forecast(steps=len(test_v))))
mape_ets_v = mean_absolute_percentage_error(test_v, model_ets_visitor.forecast(steps=len(test_v)))

# Pilih model terbaik
best_model_visitor = "ARIMA" if mape_arima_v < mape_ets_v else "ETS"

In [ ]:
# Forecast ulang untuk 30 hari ke depan menggunakan model terbaik
if best_model_visitor == "ARIMA":
    forecast = np.exp(model_arima_visitor.forecast(steps=30))
else:
    forecast = model_ets_visitor.forecast(steps=30)

forecast.index = pd.date_range(start='2019-04-01', periods=30, freq='D')
visitor_forecast_df = forecast.to_frame().reset_index()
visitor_forecast_df.columns = ['Date', 'Forecasted_Visitor']
visitor_forecast_df.head()

,Date,Forecasted_Visitor
0,2019-04-01,329.459575
1,2019-04-02,326.572321
2,2019-04-03,330.374425
3,2019-04-04,331.509607
4,2019-04-05,331.827510


5. EVALUASI & INTERPRETASI

Prediksi menunjukkan fluktuasi kunjungan harian selama April dengan lonjakan potensial di minggu kedua dan ketiga

Jumlah transaksi cenderung mengikuti tren visitor, namun dengan variabilitas yang lebih tajam pada hari-hari tertentu (mungkin karena promo atau weekend effect)

Promosi Terarah : Jika visitor meningkat di minggu kedua, tim bisa menyiapkan promosi flash sale di awal minggu agar konversi ikut naik

Manajemen Stok : Prediksi lonjakan transaksi bisa jadi alarm untuk memastikan stok produk tidak habis saat traffic tinggi

Evaluasi Model Bisnis : Jika model ETS lebih akurat (MAPE lebih kecil), maka tren dan musiman memengaruhi bisnis secara konsisten—tim bisa manfaatkan pola mingguan untuk strategi pemasaran

Dampak Operasional

Customer service dan logistik perlu bersiap di tanggal dengan proyeksi visitor atau transaksi tertinggi

Jadwal kampanye iklan sebaiknya disesuaikan dengan tanggal prediksi revenue atau visitor puncak

INTERPRETASI

Dari prediksi selama April, terlihat tren pengunjung meningkat di minggu kedua dan ketiga. Tim bisnis disarankan memanfaatkan momen ini dengan promosi bundling agar konversi optimal. Model ETS dipilih karena prediksi lebih akurat (MAPE 8.14%) dibanding ARIMA (MAPE 14.89%).”

## Dokumentasi dan Pelaporan

Proyek ini melakukan analisis penjualan sepanjang tahun 2019 untuk mendukung strategi tim bisnis dan marketing melalui eksplorasi data, pembersihan, analisis tren serta pembuatan model forecasting

### Langkah-Langkah Analisis
- Data dari Januari hingga Desember digabung dan dibersihkan dari missing values dan format tidak valid
- Revenue dihitung dari Quantity Ordered dan Price Each
- Dilakukan analisis tren harian, mingguan, dan bulanan untuk mengetahui pola belanja customer termasuk peak revenue di Desember dan jam sibuk pukul 19:00
- Produk-produk dengan kontribusi revenue terbesar di Q4 diidentifikasi sebagai dasar bundling
- Kota dengan order dan spending tertinggi dianalisis untuk strategi distribusi

### Model Forecasting
- Dua model dibangun : ARIMA dan ETS
- ETS memiliki akurasi lebih baik dengan nilai MAPE lebih kecil
- Forecast dilakukan untuk 30 hari ke depan menggunakan model terbaik

### Rekomendasi Bisnis
- Fokus promosi dan push notification di jam sibuk (19:00 – 20:00)
- Bundling produk Macbook Pro, iPhone, dan Airpods di akhir tahun
- Optimalkan strategi distribusi di kota San Francisco dan Los Angeles


Total Revenue & Statistik Transaksi

In [ ]:
# Hitung total revenue sepanjang tahun
total_annual_revenue = df_clean['Revenue'].sum()

# Hitung jumlah order sepanjang tahun (jumlah Order ID unik)
total_annual_orders = df_clean['Order ID'].nunique()

# Hitung jumlah barang yang terjual sepanjang tahun (sum dari Quantity Ordered)
total_annual_quantity = df_clean['Quantity Ordered'].sum()

# Hitung rata-rata jumlah barang yang dibeli per transaksi
average_items_per_order = total_annual_quantity / total_annual_orders

# Hitung rata-rata spending per transaksi
average_spending_per_order = total_annual_revenue / total_annual_orders

print(f"Total Revenue Sepanjang Tahun 2019: Rp {total_annual_revenue:,.2f}")
print(f"Total Jumlah Order Sepanjang Tahun 2019: {total_annual_orders:,}")
print(f"Total Jumlah Barang Terjual Sepanjang Tahun 2019: {total_annual_quantity:,.0f}")
print(f"Rata-rata Jumlah Barang per Transaksi: {average_items_per_order:.2f}")
print(f"Rata-rata Spending per Transaksi: Rp {average_spending_per_order:,.2f}")

Total Revenue Sepanjang Tahun 2019: Rp 34,492,035.97
Total Jumlah Order Sepanjang Tahun 2019: 178,437
Total Jumlah Barang Terjual Sepanjang Tahun 2019: 209,079
Rata-rata Jumlah Barang per Transaksi: 1.17
Rata-rata Spending per Transaksi: Rp 193.30


Agregasi Order Mingguan

In [ ]:
# Hitung jumlah order per minggu
# Gunakan df_clean yang sudah ada kolom 'week'
weekly_orders = df_clean.groupby('week')['Order ID'].nunique()

# Tampilkan hasil
print('Jumlah Order per Minggu:')
print(weekly_orders)

Jumlah Order per Minggu:
week
1     3290
2     2104
3     2104
4     2134
5     2381
6     2832
7     2926
8     2880
9     3075
10    3297
11    3299
12    3311
13    3221
14    4040
15    4174
16    4127
17    4014
18    3669
19    3584
20    3651
21    3585
22    3399
23    3012
24    3091
25    2967
26    3040
27    3117
28    3103
29    3150
30    3100
31    2732
32    2622
33    2577
34    2576
35    2641
36    2558
37    2686
38    2600
39    2622
40    4099
41    4341
42    4343
43    4510
44    4188
45    3963
46    3850
47    3997
48    4115
49    5317
50    5477
51    5472
52    5474
Name: Order ID, dtype: int64


Agregasi Order Bulanan

In [ ]:
# Hitung jumlah order per bulan
# Gunakan df_clean yang sudah ada kolom 'month'
# Pastikan urutan bulan sesuai
from calendar import month_name
ordered_months = list(month_name)[1:]
monthly_orders = df_clean.groupby('month')['Order ID'].nunique().reindex(ordered_months)

# Tampilkan hasil
print('Jumlah Order per Bulan:')
print(monthly_orders)

Jumlah Order per Bulan:
month
January       9293
February     11496
March        14549
April        17528
May          15836
June         12989
July         13761
August       11484
September    11202
October      19436
November     16859
December     24004
Name: Order ID, dtype: int64
